In [1]:
from numpy.fft import fft
import numpy as np
import random
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

2024-02-07 16:41:32.599900: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-07 16:41:32.599952: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-07 16:41:32.601616: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-07 16:41:32.760362: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
N_data = 2500  # Number of training data in final augmented dataset
L = 1024      # Length of IQ sequence in each sample
D = 100 # Decimation factor for long-term data
SNRdBmin = 10
SNRdBmax = 20

max_pw  = 1000 # Pulse Width (us)
max_pri = 1000 # Pulse Repetition Interval (us)
min_sig = 0.01
max_sig = 1    # Magnitude of the radar signal
SNRdBmin = 0
SNRdBmax = 20
fs = 20e6
LFM_maxBW = 10e6
LFM_minBW = 1e6
max_agility_BW = 5e6
min_agility_BW = 1e6
seq_len = 1000

modtype_checkpoint_filepath = 'temp/checkpoint/modtype_2.h5'
freqagility_checkpoint_filepath = 'temp/checkpoint/freq_agility_2.h5'
pwagility_checkpoint_filepath = 'temp/checkpoint/pw_agility_2.h5'
priagility_checkpoint_filepath = 'temp/checkpoint/pri_agility_2.h5'
pw_checkpoint_filepath = 'temp/checkpoint/pw_2.h5'
pri_checkpoint_filepath = 'temp/checkpoint/pri_2.h5'

log_filepath = 'temp/tb_logs'

N_train_iter = 100

In [3]:
def gen_data():
    data = np.zeros((N_data, L, 8))

    # 9 columns: 
    #   0: modulation (0=unmodulated, 1=linear frequency modulation) 
    #   1: frequency agility (0=no agility, 1=jitter)
    #   2: pulse width agility (0=no agility, 1=jitter)
    #   3: pulse repetition interval agility (0=no agility,1=jitter)
    #   4: minimum pulse width
    #   5: maximum pulse width (same as minimum if no agility)
    #   6: minimum pulse repetition interval
    #   7: maximum pulse repetition interval (same as minimum if no agility)
    #   8: SNR in dB
    labels = np.zeros((N_data, 9)) 

    for i in range(N_data):
        # Select Radar emitter parameters
        pri_min = random.uniform(0, np.log10(max_pri)) 
        pri_min = 10**pri_min
        pw_max = random.uniform(0, np.log10(min(pri_min, max_pw))) # PW needs to be less than pri, or the radar will be CW
        pw_max = 10**pw_max
        sig_mag = random.uniform(np.log10(min_sig), np.log10(max_sig))
        sig_mag = 10**sig_mag
        phase_off = random.uniform(0, 2*np.pi)

        pulse_phase = random.uniform(0, 1) # Phase between 0 and 1 as percent of cycle elapsed at start of waveform
        mod_type = random.randint(0,1) # 0 is unmodulated, 1 is LFM
        freq_agile = random.randint(0,1) # 0 is no agility, 1 is jitter
        pw_agile = random.randint(0,1) # 0 is no agility, 1 is jitter
        pri_agile = random.randint(0,1) # 0 is no agility, 1 is jitter
        freq_agility_BW = random.uniform(min_agility_BW, max_agility_BW)
        LFM_BW = random.uniform(np.log10(LFM_maxBW), np.log10(LFM_minBW))
        LFM_BW = 10**LFM_BW

        freq_seq = np.zeros(seq_len)
        freq_off_seq = np.zeros(seq_len)
        if (freq_agile == 1):
            for j in range(seq_len):
                freq_seq[j] = random.uniform(-LFM_BW/2, LFM_BW/2) # deviation of frequency from center frequency
                freq_off_seq[j] = random.uniform(0, 2*np.pi) # phase offset that varies per pulse

        pw_seq = np.zeros(seq_len)
        if (pw_agile == 1):
            pw_min = random.uniform(np.log10(1), np.log10(pw_max))
            pw_min = 10**pw_min
            for j in range(seq_len):
                pw_seq[j] = random.uniform(pw_min, pw_max) 
        else:
            pw_min = pw_max

        pri_seq = np.zeros(seq_len)
        if (pri_agile == 1):
            pri_max = random.uniform(np.log10(pri_min), np.log10(max_pri))
            pri_max = 10**pri_max
            for j in range(seq_len):
                pri_seq[j] = random.uniform(pri_min, pri_max)
        else:
            pri_max = pri_min


        t = np.linspace(0.0, (L*D-1)/fs, L*D)
        dt = t[1]-t[0]

        x_mag = np.zeros(D*L)
        x_phase = np.zeros(D*L)
        # No modulation or agility
        if (mod_type == 0) & (pw_agile == 0) & (pri_agile == 0) & (freq_agile == 0):
            x_mag = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) < (pw_min*1e-6)
            x_phase = phase_off
        # basic LFM
        elif (mod_type ==1) & (pw_agile == 0) & (pri_agile == 0) & (freq_agile == 0):
            x_mag   = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) < (pw_min*1e-6)
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) * x_mag # This should represent the time elapsed since start of the pulse
            x_phase = np.pi*LFM_BW*(x_time**2/(pw_min*1e-6)-x_time)
        # unmod freq_agile
        elif (mod_type ==0) & (freq_agile == 1) & (pw_agile == 0) & (pri_agile == 0):
            x_mag   = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) < (pw_min*1e-6)
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) * x_mag # This should represent the time elapsed since start of the pulse
            k = (t // (pri_min*1e-6)) # This should represent which pulse we are on
            k = k.astype(int) # Turn the floats in the array into integers so that they can be used for indexing. Lord only knows why integer division returns a float
            x_phase = 2*np.pi*freq_seq[k % seq_len]*x_time+freq_off_seq[k % seq_len]
        # LFM freq_agile
        elif (mod_type ==1) & (freq_agile == 1) & (pw_agile == 0) & (pri_agile == 0):
            x_mag   = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) < (pw_min*1e-6)
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) * x_mag # This should represent the time elapsed since start of the pulse
            k = (t // (pri_min*1e-6)) # This should represent which pulse we are on
            k = k.astype(int)
            x_phase = np.pi*LFM_BW*(x_time**2/(pw_min*1e-6)-x_time)+2*np.pi*freq_seq[k % seq_len]*x_time+freq_off_seq[k % seq_len]
        # unmod pw_agile
        elif (mod_type == 0) & (freq_agile == 0) & (pw_agile == 1) & (pri_agile == 0):
            k = (t // (pri_min*1e-6)) # This should represent which pulse we are on
            k = k.astype(int)
            x_mag = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) < (pw_seq[k % seq_len]*1e-6)
            x_phase = phase_off
        # LFM pw_agile
        elif (mod_type == 1) & (freq_agile == 0) & (pw_agile == 1) & (pri_agile == 0):
            k = (t // (pri_min*1e-6)) # This should represent which pulse we are on
            k = k.astype(int)
            x_mag = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) < (pw_seq[k % seq_len]*1e-6)
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) * x_mag # This should represent the time elapsed since start of the pulse
            x_phase = np.pi*LFM_BW*(x_time**2/(pw_seq[k % seq_len]*1e-6)-x_time)
        # unmod pw_agile freq_agile
        elif (mod_type == 0) & (freq_agile == 1) & (pw_agile == 1) & (pri_agile == 0):
            k = (t // (pri_min*1e-6)) # This should represent which pulse we are on
            k = k.astype(int)
            x_mag = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) < (pw_seq[k % seq_len]*1e-6)
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) * x_mag # This should represent the time elapsed since start of the pulse
            x_phase = 2*np.pi*freq_seq[k % seq_len]*x_time+freq_off_seq[k % seq_len]
        # LFM pw_agile freq_agile
        elif (mod_type == 1) & (freq_agile == 1) & (pw_agile == 1) & (pri_agile == 0):
            k = (t // (pri_min*1e-6)) # This should represent which pulse we are on
            k = k.astype(int)
            x_mag = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) < (pw_seq[k % seq_len]*1e-6)
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) * x_mag # This should represent the time elapsed since start of the pulse
            x_phase = np.pi*LFM_BW*(x_time**2/(pw_seq[k % seq_len]*1e-6)-x_time) + 2*np.pi*freq_seq[k % seq_len]*x_time+freq_off_seq[k % seq_len]
        # pri_agile
        elif (mod_type == 0) & (freq_agile == 0) & (pw_agile == 0) & (pri_agile == 1):
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) # Time since start of signal
            x_time_p = np.zeros(D*L) # Time since start of pulse
            k = np.zeros(D*L,dtype=int)
            for j in range(1,D*L):
                if (x_time_p[j-1] >= 1e-6*pri_seq[k[j-1]%seq_len]):
                    k[j] = k[j-1] + 1
                    x_time_p[j] = 0
                else:
                    k[j] = k[j-1]
                    x_time_p[j] = x_time_p[j-1] + dt
            x_mag = x_time_p < (pw_min*1e-6)
            x_phase = phase_off
        # LFM pri_agile
        elif (mod_type == 1) & (freq_agile == 0) & (pw_agile == 0) & (pri_agile == 1):
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) # Time since start of signal
            x_time_p = np.zeros(D*L) # Time since start of pulse
            k = np.zeros(D*L,dtype=int)
            for j in range(1,D*L):
                if (x_time_p[j-1] >= 1e-6*pri_seq[k[j-1]%seq_len]):
                    k[j] = k[j-1] + 1
                    x_time_p[j] = 0
                else:
                    k[j] = k[j-1]
                    x_time_p[j] = x_time_p[j-1] + dt
            x_mag = x_time_p < (pw_min*1e-6)
            x_phase = np.pi*LFM_BW*(x_time_p**2/(pw_min*1e-6)-x_time_p)
        # freq_agile pri_agile
        elif (mod_type == 0) & (freq_agile == 1) & (pw_agile == 0) & (pri_agile == 1):
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) # Time since start of signal
            x_time_p = np.zeros(D*L) # Time since start of pulse
            k = np.zeros(D*L,dtype=int)
            for j in range(1,D*L):
                if (x_time_p[j-1] >= 1e-6*pri_seq[k[j-1]%seq_len]):
                    k[j] = k[j-1] + 1
                    x_time_p[j] = 0
                else:
                    k[j] = k[j-1]
                    x_time_p[j] = x_time_p[j-1] + dt
            x_mag = x_time_p < (pw_min*1e-6)
            x_phase = 2*np.pi*freq_seq[k % seq_len]*x_time_p+freq_off_seq[k % seq_len]
        # LFM freq_agile pri_agile
        elif (mod_type == 1) & (freq_agile == 1) & (pw_agile == 0) & (pri_agile == 1):
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) # Time since start of signal
            x_time_p = np.zeros(D*L) # Time since start of pulse
            k = np.zeros(D*L,dtype=int)
            for j in range(1,D*L):
                if (x_time_p[j-1] >= 1e-6*pri_seq[k[j-1]%seq_len]):
                    k[j] = k[j-1] + 1
                    x_time_p[j] = 0
                else:
                    k[j] = k[j-1]
                    x_time_p[j] = x_time_p[j-1] + dt
            x_mag = x_time_p < (pw_min*1e-6)
            x_phase = np.pi*LFM_BW*(x_time_p**2/(pw_min*1e-6)-x_time_p)+2*np.pi*freq_seq[k % seq_len]*x_time_p+freq_off_seq[k % seq_len]
        # pw_agile pri_agile
        elif (mod_type == 0) & (freq_agile == 0) & (pw_agile == 1) & (pri_agile == 1):
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) # Time since start of signal
            x_time_p = np.zeros(D*L) # Time since start of pulse
            k = np.zeros(D*L,dtype=int)
            for j in range(1,D*L):
                if (x_time_p[j-1] >= 1e-6*pri_seq[k[j-1]%seq_len]):
                    k[j] = k[j-1] + 1
                    x_time_p[j] = 0
                else:
                    k[j] = k[j-1]
                    x_time_p[j] = x_time_p[j-1] + dt
            x_mag = x_time_p < (pw_seq[k % seq_len]*1e-6)
            x_phase = phase_off
        # LFM pw_agile pri_agile
        elif (mod_type == 1) & (freq_agile == 0) & (pw_agile == 1) & (pri_agile == 1):
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) # Time since start of signal
            x_time_p = np.zeros(D*L) # Time since start of pulse
            k = np.zeros(D*L,dtype=int)
            for j in range(1,D*L):
                if (x_time_p[j-1] >= 1e-6*pri_seq[k[j-1]%seq_len]):
                    k[j] = k[j-1] + 1
                    x_time_p[j] = 0
                else:
                    k[j] = k[j-1]
                    x_time_p[j] = x_time_p[j-1] + dt
            x_mag = x_time_p < (pw_seq[k % seq_len]*1e-6)
            x_phase = np.pi*LFM_BW*(x_time_p**2/(pw_seq[k % seq_len]*1e-6)-x_time_p)
        # freq_agile pw_agile pri_agile
        elif (mod_type == 0) & (freq_agile == 1) & (pw_agile == 1) & (pri_agile == 1):
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) # Time since start of signal
            x_time_p = np.zeros(D*L) # Time since start of pulse
            k = np.zeros(D*L,dtype=int)
            for j in range(1,D*L):
                if (x_time_p[j-1] >= 1e-6*pri_seq[k[j-1]%seq_len]):
                    k[j] = k[j-1] + 1
                    x_time_p[j] = 0
                else:
                    k[j] = k[j-1]
                    x_time_p[j] = x_time_p[j-1] + dt
            x_mag = x_time_p < (pw_seq[k % seq_len]*1e-6)
            x_phase = 2*np.pi*freq_seq[k % seq_len]*x_time_p+freq_off_seq[k % seq_len]
        # LFM freq_agile pw_agile pri_agile
        elif (mod_type == 1) & (freq_agile == 1) & (pw_agile == 1) & (pri_agile == 1):
            x_time = ((t + pulse_phase*pri_min) % (pri_min*1e-6)) # Time since start of signal
            x_time_p = np.zeros(D*L) # Time since start of pulse
            k = np.zeros(D*L,dtype=int)
            for j in range(1,D*L):
                if (x_time_p[j-1] >= 1e-6*pri_seq[k[j-1]%seq_len]):
                    k[j] = k[j-1] + 1
                    x_time_p[j] = 0
                else:
                    k[j] = k[j-1]
                    x_time_p[j] = x_time_p[j-1] + dt
            x_mag = x_time_p < (pw_seq[k % seq_len]*1e-6)
            x_phase = np.pi*LFM_BW*(x_time_p**2/(pw_min*1e-6)-x_time_p)+2*np.pi*freq_seq[k % seq_len]*x_time_p+freq_off_seq[k % seq_len]
        

        x = x_mag*np.exp(1j*x_phase)
        x = sig_mag*x

        # Add AWGN noise with random power
        SNRdB = random.uniform(SNRdBmin, SNRdBmax)
        SNR = 10**(SNRdB/10)
        sig_pwr = np.mean(np.abs(x)**2)
        sigma2 = sig_pwr/SNR
        noise = np.random.randn(np.size(x))*np.sqrt(sigma2/2) + 1j*np.random.randn(np.size(x))*np.sqrt(sigma2/2)
        x_long = x + noise

        x_short = x_long[np.arange(0, int(np.size(x_long)/D), 1)]
        x_long = x_long[np.arange(0, np.size(x_long), D)]

        xf_short = fft(x_short)
        xf_long  = fft(x_long)


        x_ms = abs(x_short)
        x_ps = np.angle(x_short)
        x_ml = abs(x_long)
        x_pl = np.angle(x_long)
        xf_ms = abs(xf_short)
        xf_ps = np.angle(xf_short)
        xf_ml = abs(xf_long)
        xf_pl = np.angle(xf_long)

        data[i,:,0] = x_ms
        data[i,:,1] = x_ps
        data[i,:,2] = x_ml
        data[i,:,3] = x_pl
        data[i,:,4] = xf_ms
        data[i,:,5] = xf_ps
        data[i,:,6] = xf_ml
        data[i,:,7] = xf_pl

        labels[i,0] = mod_type
        labels[i,1] = freq_agile
        labels[i,2] = pw_agile
        labels[i,3] = pri_agile
        labels[i,4] = pw_min
        labels[i,5] = pw_max
        labels[i,6] = pri_min
        labels[i,7] = pri_max
        labels[i,8] = SNRdB

    return data, labels

In [4]:
def build_classifier_model(input_shape,output_size):
    model = Sequential()
    model.add(Conv1D(10, kernel_size=5, padding='same', input_shape=input_shape, activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(20, kernel_size=5, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(30, kernel_size=5, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(40, kernel_size=5, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(50, kernel_size=5, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(60, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(70, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(80, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(90, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Flatten())
    model.add(Dense(200, activation='relu'))
    model.add(Dense(output_size, activation='softmax'))
    return model

In [5]:
def build_regressor_model(input_shape,output_size):
    model = Sequential()
    model.add(Conv1D(10, kernel_size=5, padding='same', input_shape=input_shape, activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(20, kernel_size=5, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(30, kernel_size=5, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(40, kernel_size=5, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(50, kernel_size=5, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(60, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(70, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(80, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(90, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Flatten())
    model.add(Dense(200, activation='relu'))
    model.add(Dense(output_size))
    return model

In [6]:
model_pwagility = build_classifier_model(input_shape=(L,8),output_size=2)
model_modtype = build_classifier_model(input_shape=(L,8),output_size=2)
model_priagility = build_classifier_model(input_shape=(L,8),output_size=2)
model_freqagility = build_classifier_model(input_shape=(L,8),output_size=2)
model_pw = build_regressor_model(input_shape=(L,8),output_size=2)
model_pri = build_regressor_model(input_shape=(L,8),output_size=2)

2024-02-07 16:41:36.429192: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-07 16:41:36.458623: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-07 16:41:36.458667: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-07 16:41:36.461534: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-07 16:41:36.461579: I tensorflow/compile

In [7]:
# Compile the model
model_modtype.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
model_freqagility.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
model_pwagility.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
model_priagility.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
model_pw.compile(loss='mse', optimizer='Adam')
model_pri.compile(loss='mse', optimizer='Adam')
# Setup the callbacks
checkpt_modtype = ModelCheckpoint(modtype_checkpoint_filepath, save_best_only=True, verbose=0)
checkpt_freqagility = ModelCheckpoint(freqagility_checkpoint_filepath, save_best_only=True, verbose=0)
checkpt_pwagility = ModelCheckpoint(pwagility_checkpoint_filepath, save_best_only=True, verbose=0)
checkpt_priagility = ModelCheckpoint(priagility_checkpoint_filepath, save_best_only=True, verbose=0)
checkpt_pw = ModelCheckpoint(pw_checkpoint_filepath, save_best_only=True, verbose=0)
checkpt_pri = ModelCheckpoint(pri_checkpoint_filepath, save_best_only=True, verbose=0)
tb = TensorBoard(log_dir=log_filepath)
e_stop = EarlyStopping(patience=5)

In [9]:
x, labels = gen_data()
y_pwmin = labels[:,4]
y_pwmax = labels[:,5]
y_primin = labels[:,6]
y_primax = labels[:,7]

b_pwmin = np.mean(y_pwmin)
m_pwmin = max(abs(y_pwmin-b_pwmin))
b_pwmax = np.mean(y_pwmax)
m_pwmax = max(abs(y_pwmax-b_pwmax))
b_primin = np.mean(y_primin)
m_primin = max(abs(y_primin-b_primin))
b_primax = np.mean(y_primax)
m_primax = max(abs(y_primax-b_primax))
print(max((y_pwmin-b_pwmin)/m_pwmin), min((y_pwmin-b_pwmin)/m_pwmin))
print((y_pwmin-b_pwmin)/m_pwmin)

1.0 -0.017740782871054107
[-0.01567626 -0.01466481 -0.01625123 ...  0.31469235 -0.01553876
 -0.0170214 ]


In [22]:
print('pwmin  b:', b_pwmin,  ' m:', m_pwmin)
print('pwmax  b:', b_pwmax,  ' m:', m_pwmax)
print('primin b:', b_primin, ' m:', m_primin)
print('primax b:', b_primax, ' m:', m_primax)

pwmin  b: 15.286275640628888  m: 805.2782390280091
pwmax  b: 25.039697983513843  m: 795.5248166851242
primin b: 150.03263307411285  m: 842.559115216757
primax b: 260.4717819805048  m: 738.0919126349022


In [20]:
for i in range(N_train_iter):
    print(i)
    x, labels = gen_data()
    y_modtype = labels[:,0] # Modulation data
    y_freqagility = labels[:,1] # freq agility data
    y_pwagility = labels[:,2]
    y_priagility = labels[:,3]
    y_pw = labels[:,4:6]
    y_pri = labels[:,6:8]

    y_pw[:,0] = (y_pw[:,0]-b_pwmin)/m_pwmin
    y_pw[:,1] = (y_pw[:,1]-b_pwmax)/m_pwmax

    y_pri[:,0] = (y_pri[:,0]-b_primin)/m_primin
    y_pri[:,1] = (y_pri[:,1]-b_primax)/m_primax
    
    # Train the models
    print('MODULATION TYPE')
    hist = model_modtype.fit(x, y_modtype, epochs=100, validation_split=0.2, verbose=2, batch_size=50,
                    callbacks=[checkpt_modtype, tb, e_stop])
    print('FREQUENCY AGILITY')
    hist = model_freqagility.fit(x, y_freqagility, epochs=100, validation_split=0.2, verbose=2, batch_size=50,
                    callbacks=[checkpt_freqagility, tb, e_stop])
    print('PW AGILITY')
    hist = model_pwagility.fit(x, y_pwagility, epochs=100, validation_split=0.2, verbose=2, batch_size=50,
                    callbacks=[checkpt_pwagility, tb, e_stop])
    print('PRI AGILITY')
    hist = model_priagility.fit(x, y_priagility, epochs=100, validation_split=0.2, verbose=2, batch_size=50,
                    callbacks=[checkpt_priagility, tb, e_stop])
    print('PW')
    hist = model_pw.fit(x, y_pw, epochs=100, validation_split=0.2, verbose=2, batch_size=50,
                    callbacks=[checkpt_pw, tb, e_stop])
    print('PRI')
    hist = model_pri.fit(x, y_pw, epochs=100, validation_split=0.2, verbose=2, batch_size=50,
                    callbacks=[checkpt_pri, tb, e_stop])
    

0
MODULATION TYPE
Epoch 1/100
40/40 - 1s - loss: 0.5052 - accuracy: 0.7545 - val_loss: 0.4953 - val_accuracy: 0.7420 - 899ms/epoch - 22ms/step
Epoch 2/100
40/40 - 1s - loss: 0.3965 - accuracy: 0.8200 - val_loss: 0.4022 - val_accuracy: 0.7900 - 721ms/epoch - 18ms/step
Epoch 3/100


/home/thomas/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


40/40 - 1s - loss: 0.3014 - accuracy: 0.8630 - val_loss: 0.4042 - val_accuracy: 0.8040 - 615ms/epoch - 15ms/step
Epoch 4/100
40/40 - 1s - loss: 0.2552 - accuracy: 0.8985 - val_loss: 0.4698 - val_accuracy: 0.7720 - 586ms/epoch - 15ms/step
Epoch 5/100
40/40 - 1s - loss: 0.1870 - accuracy: 0.9265 - val_loss: 0.4993 - val_accuracy: 0.7980 - 599ms/epoch - 15ms/step
Epoch 6/100
40/40 - 1s - loss: 0.1517 - accuracy: 0.9475 - val_loss: 0.4659 - val_accuracy: 0.8000 - 593ms/epoch - 15ms/step
Epoch 7/100
40/40 - 1s - loss: 0.1312 - accuracy: 0.9490 - val_loss: 0.6567 - val_accuracy: 0.7820 - 573ms/epoch - 14ms/step
FREQUENCY AGILITY
Epoch 1/100
40/40 - 1s - loss: 0.4876 - accuracy: 0.7605 - val_loss: 0.4938 - val_accuracy: 0.7380 - 714ms/epoch - 18ms/step
Epoch 2/100
40/40 - 1s - loss: 0.3948 - accuracy: 0.8200 - val_loss: 0.4932 - val_accuracy: 0.7520 - 727ms/epoch - 18ms/step
Epoch 3/100
40/40 - 1s - loss: 0.3228 - accuracy: 0.8515 - val_loss: 0.4396 - val_accuracy: 0.7900 - 756ms/epoch - 19ms

2024-02-07 17:55:31.519241: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 218.8KiB (rounded to 224000)requested by op sequential_4/conv1d_42/Conv1D-0-0-TransposeNCHWToNHWC-LayoutOptimizer
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-02-07 17:55:31.519361: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2024-02-07 17:55:31.519373: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 2414, Chunks in use: 2413. 603.5KiB allocated for chunks. 603.2KiB in use in bin. 24.8KiB client-requested in use in bin.
2024-02-07 17:55:31.519379: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 59, Chunks in use: 59. 29.5KiB allocated for chunks. 29.5KiB in use in bin. 18.8KiB client-requested in 

ResourceExhaustedError: Graph execution error:

Detected at node sequential_4/conv1d_42/Conv1D-0-0-TransposeNCHWToNHWC-LayoutOptimizer defined at (most recent call last):
<stack traces unavailable>
OOM when allocating tensor with shape[50,1,16,70] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node sequential_4/conv1d_42/Conv1D-0-0-TransposeNCHWToNHWC-LayoutOptimizer}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_51466]

In [19]:
labels[:,4:9].shape

(2500, 5)

In [15]:
y_pw.shape

(2500, 1)

SyntaxError: illegal target for annotation (2616603510.py, line 1)